### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute.

In [1]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content="<think>\nOkay, so I need to figure out why parrots talk. Let me start by recalling what I know. Parrots are known for mimicking human speech, right? But why do they do that? Maybe it's a way they communicate, like how some animals use sounds. I remember reading that parrots are highly intelligent birds, so maybe talking is part of their learning behavior.\n\nWait, in the wild, do parrots talk? Or is that something they learn in captivity? Maybe they imitate other birds or sounds in their environment. So in the wild, they might mimic other animals or natural sounds. In captivity, they mimic humans because they're around people. But why would they imitate sounds? Could it be for social interaction? Like, if they're trying to communicate with their owners to get food or attention?\n\nI've heard that some parrots talk because they want to interact. Maybe it's a form of social bonding. They're social animals, so imitating their human's speech could be a way to engage with

In [3]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [5]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': "Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. It requires a location, which is Boston here. I'll call the function with location set to Boston. Make sure the JSON is correctly formatted with the name and arguments.\n", 'tool_calls': [{'id': 'ftqq1n6qe', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 154, 'total_tokens': 240, 'completion_time': 0.147784971, 'completion_tokens_details': {'reasoning_tokens': 62}, 'prompt_time': 0.006115851, 'prompt_tokens_details': None, 'queue_time': 0.157721539, 'total_time': 0.153900822}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019bcc37-2325-76c1-b

### Tool Execution Loops

In [4]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is sunny.


In [5]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. I need to use the get_weather function. Let me check the function parameters. The required parameter is location, which should be a string. Boston is the location here. So I\'ll call the function with location set to "Boston". Make sure the JSON is correctly formatted with the function name and arguments. No other functions are available, so this should be straightforward.\n', 'tool_calls': [{'id': '20cfda4p0', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 153, 'total_tokens': 262, 'completion_time': 0.199966855, 'completion_tokens_details': {'reasoning_tokens': 85}, 'prompt_time': 0.006519155, 'prompt_tokens_details': None, 'queue_time': 0.055720395, 'total_time': 0.20648601}, 'mod